<a href="https://colab.research.google.com/github/NamalSanjaya/CV_A04/blob/main/Copy_of_A04_Q02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [ ]:
x_train_copy , x_test_copy = x_train.copy() , x_test.copy()

In [ ]:
K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]

Nte = x_test.shape[0]
Din = 3072 # CIFAR10
# Din = 784 # MINIST

# Normalize pixel values

x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
x_test = x_test - mean_image

y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)
x_train = np.reshape(x_train,(Ntr,Din))
x_test = np.reshape(x_test,(Nte,Din))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')



In [ ]:
std=1e-5
H=200
w1 = std*np.random.randn(Din, H)
b1 = np.zeros(H)
w2 = std*np.random.randn(H, K)
b2 = np.zeros(K)

batch_size = Ntr
iterations = 300
lr = 1.4e-2
lr_decay= 0.999
reg = 5e-6
loss_history = []
train_acc_history = []
val_acc_history = []
seed = 0
#rng = np.random.default_rng(seed=seed)



In [ ]:
def trainAcc(true,predict):
  return 1.0 - 1.0/(Ntr*K)*np.abs(np.argmax(true , axis=1) - np.argmax(predict , axis=1) ).sum()


In [ ]:
def backprogation(true,predict,H):

  dL_dyt = 1./batch_size*2.0*(predict-true)
  dw2     = H.T.dot(dL_dyt) + reg*w2
  db2     = dL_dyt.sum(axis=0)
  dh      = dL_dyt.dot(w2.T)
  dw1     = x.T.dot(dh * H*(1-H)) +  reg*w1
  db1     =  (dh * H*(1-H)).sum(axis=0)

  return dw1,db1,dw2,db2


In [ ]:

## Batch Gradient descent 

for t in range(iterations):
  batch_indices = np.random.choice(Ntr,batch_size)
  x = x_train[batch_indices]
  y = y_train[batch_indices]
  h = 1.0/(1.0 + np.exp( -(x.dot(w1) + b1) ))
  y_pred = h.dot(w2) + b2
  loss  = 1./batch_size*np.square(y_pred - y).sum()  + reg*(np.sum(w1*w1) + np.sum(w2*w2)) 
  loss_history.append(loss)

  if t%50 == 0:
    Acc = trainAcc(y, y_pred)
    print("Loss : " , loss , " | Accuray :", Acc)

  dw1,db1,dw2,db2 = backprogation(y,y_pred,h)

  w1 -= lr*dw1
  b1 -= lr*db1
  w2 -= lr*dw2
  b2 -= lr*db2
  lr *= lr_decay


Loss :  1.0000486724497226  | Accuray : 0.62956
Loss :  0.9000173940475015  | Accuray : 0.630804
Loss :  0.9000320669726078  | Accuray : 0.6907300000000001
Loss :  0.900013015072964  | Accuray : 0.73064
Loss :  0.9000162795573512  | Accuray : 0.549656
Loss :  0.9000445101453538  | Accuray : 0.729762


In [ ]:
 ### calculating testing accuracy

hn            =  1.0/(1.0 + np.exp( -(x_test.dot(w1) + b1) ))
predictY     =  hn.dot(w2) + b2
test_loss    = 1./batch_size*np.square(predictY - y_test).sum()  + reg*(np.sum(w1*w1) + np.sum(w2*w2))
test_accuracy =  1.0 - 1.0/(1e5)*np.abs(np.argmax(y_test , axis=1) - np.argmax(predictY , axis=1) ).sum()

print("loss : " , test_loss , "Accuracy : " , test_accuracy)